# AES

In [282]:
from IPython.display import display, Math, Latex
import functools
import itertools

## The Galois field $GF(2^8)$

AUS uses field arithmetic in $GF(2^8)$, represented by polynomials modulo the irreducible $x^8 + x^4+ x^3+x+1$. That is, coefficients are in $\mathbb{Z}_2$, so they can be stored as bits. The polynomials in the field all have degree less than $8$. So they can be stored as bytes, and arithmetic is all implemented as byte operations.

In any Galois field $GF(p^k)$, addition is just polynomial addition with coefficients in $\mathbb{Z}_k$. For AES, this addition in $\mathbb{Z}_2$ is just exclusive or. For example, in Python ```^``` is xor.


In [260]:
hex(0x15 ^ 0x24)

'0x31'

Define the hex value corresponding to our irreducible polynomial.

In [261]:
m = 0x11b

How do you know ```0x11b``` is correct? This write a little helper to render bit strings has polynomials

In [262]:
def x_term(d):
    if d == 0:
        return "1"
    elif d == 1:
        return "x"
    else:
        return "x^{%d}"%d
    
def latex_poly(p):
    if p == 0:
        return "0"

    d = 0
    while p%2 == 0:
        d += 1
        p >>= 1
    
    s = x_term(d)
    
    p >>= 1
    d += 1
    while p != 0:
        if p%2==1:
            s = "%s + %s"%(x_term(d),s)
        p >>=1
        d += 1
    return s
    

In [263]:
display(Math(latex_poly(m)))

<IPython.core.display.Math object>

Now polynomial multiplication in the ring $\mathbb{Z}_2$ is pretty simple. For $a\cdot b$,
use $b$ to determine which muplitples of $a$ should be added as partial products.

In [264]:
def Z2_mult(a,b):
    res = 0
    p = b
    while a != 0:
        if a%2 == 1:
            res ^= p
        a >>= 1 # divide by x
        p <<= 1 # multiply by x
    return res
    

In [265]:
display(Math("(%s)\cdot (%s) = %s"%(latex_poly(0x11), latex_poly(0x32), latex_poly(Z2_mult(0x11,0x32)))))

<IPython.core.display.Math object>

But we want modular multiplication

In [266]:
def AES_mult(a,b):
    res = 0
    p = b
    while a != 0:
        if a%2 == 1:
            res ^= p
        a >>= 1 # divide by x (discard constant coefficient)
        p <<= 1
        if p&0x100:
            p ^= 0x11b

    return res

In [267]:
hex(AES_mult(0x57,0x83))

'0xc1'

In [268]:
display(Math("(%s)\cdot (%s) = %s"%(latex_poly(0x57), latex_poly(0x83), latex_poly(AES_mult(0x57,0x83)))))

<IPython.core.display.Math object>

Now that we have this field worked out, we will only use it explicitly as a ring. Multiplicative inverses play a role only in the _S-box_ phases of the algorithm.

Nevertheless, we can ask whether the irreducible is primitive:

In [269]:
powers = set()
power = 1
for i in range(255):
    powers.add(power)
    # power = power*x
    power <<= 1
    if power&0x100:
        power ^= 0x11b
len(powers)

51

Nope. This would be 255 if the modulus were primitive. Oh well.

Let's compute the multiplicative inverses anyway. There are only 255 of them.

I'm lazy, and we only need to do this once. So ...

In [270]:
inv_table = {}
for i in range(1,256):
    for j in range(1,i+1):
        if AES_mult(i,j) == 0x01:
            inv_table[i]=j
            inv_table[j] = i


In [271]:
ok = True
for i in range(1,256):
    if AES_mult(i,inv_table[i]) != 1:
        ok = False
        print("Hey. This is wrong %x * %x != 1"%(i,inv_table[i]))
if ok:
    print("Life is good")

Life is good


## The polynomial ring $GF(2^8)[x]/(x^4+1)$

$GF(2^8)[x]$ denotes the _ring_ of polynomials with coefficients in $GF(2^8)$. This is only a ring, not a field. Dividing a ring by an element of that ring (in this case, a $x^4+1$) means we take all elements modulo that element. In effect, we are saying that for our purposes, $x^4+1 \equiv 0$. 

### What this actually does to elements of $GF(2^8)[x]$

Apparently, $x^4 \equiv -1 \pmod {x^4 + 1}$. But remember that the coefficients are in $GF(2^8)$,
and in that field $-1 = 1$. So actually, the important point is that $$x^4\equiv 1 \pmod {x^4 + 1}.$$

So suppose we want to reduce some element of $GF(2^8)[x]$ modulo $x^4+1$. For example, to reduce
$$a_9x^9+a_8x^8+a_7x^7+a_6x^6+a_5x^5+a_4x^4+a_3x^3+a_2x^2+a_1x+a_0,$$
we write as (apology for the format)

```
                   a_9x^9 + a_8x^8
+a_7x^7 + a_6x^6 + a_5x^5 + a_4x^4
+a_3x^3 + a_2x^2 + a_1x   + a_0
```

Addition is just xor, so we simply add columns.

Hence multiplication in the ring $GF(2^8)[x]/x^4+1$ has a very simple algorithm.



In [363]:
def AES_poly_mult(p,q):
    res = [0,0,0,0]
    for i in range(4):
        for j in range(4):
            res[(i+j)%4] ^= AES_mult(p[i],q[j])
        
    return res

An alternative that can be adapted to faster operation if poynomials are stored as ```uint32_t```.

In [365]:
def AES_poly_mult_alt(p,q):
    res = [0,0,0,0]
    for k in range(4):
        for i in range(4):
            res[k] ^= AES_mult(p[i],q[(-i)%4])
    return res

The polynomial $x^4_1$ is definitely not irreducible. **Explain ...**

Nevertheless, there are polynomials that are invertible in this ring.

The following is a test of AES_poly_mult. Result should be $(1,0,0,0)$.

In [325]:
tuple(AES_poly_mult([0x02,0x01,0x01,0x03],[0x0e,0x09,0x0d,0x0b]))

(1, 0, 0, 0)

In [326]:
tuple(AES_poly_mult([0x00,0x01,0x00,0x00],[0x0e,0x09,0x0d,0x0b]))

(11, 14, 9, 13)

The AES algorithm only employs two pairs of invertible elements:

*  $ex^3 + 9x^2+dx + b$ and $3x^3+x^2+x+2$
* $x^3$ and $x$ (remember $x^3\cdot x = x^4\equiv 1 \pmod {x^4+1}$)

The later pair simply rotates coefficients.

## The ring $\mathbb{Z}_2[x]/(x^8+1)$

The ring of polynomials over $\mathbb{Z}_2$ is also used in one place in AES. 

The coefficients share a property with all rings of polynomials over Galois fields $GF(2^k)$. Namely, $$p+p = 0.$$

So for any $k$, 

$$x^k\equiv 1\pmod {x^k+1}$$

Evidently if $k$ is even, this is not an irreducible polynomial, but we don't need it to be. This does, however, simplify multiplication just as it does for $GF(2^8)[x]/x^4+1$.

AES only uses arithmetic in the ring $\mathbb{Z}_2[x]/(x^8+1)$ in one way, so obtain an invertible affine transformation:

$$T(p) = (x^4+1)p + 0x63 \bmod {x^8+1}$$

This looks like a non-linear transformation, but remember that $x^4+1$ is really just a constant in $\mathbb{Z}_2[x]/(x^8+1)$.

So this is really affine. And it is invertible. Think about this in reals:

$$y = ax + b\quad \text{iff}\quad (y-b)/a = x.$$

The same thing works here, so long and the coefficient $a$ is invertible. In our situation,

$$(x^4+1)(x^4+1) = x^8 + 2x^4+1 \equiv 1\bmod{x^8+1}$$

So the tranformation $T$ has as its inverse:

$$T^{-1}(p) = (x^4+1)p - 0x63 \bmod {x^8+1}.$$

__Hang on!__ $-c$ and $c$ are the same. So $T = T^{-1}$. Or if you prefer, $T^2 = I$. 



The product $(x^4 + 1)p$ is really simple. We just rotate the 8 coefficients of p by 4, and add to p (that is, xor).

So $T$ is easy to code.

In [327]:
def T(p):
    return ((p&0xf)<<4 & (p>>4)&0x0f)^ p ^ 0x63

Now AES uses $T$ and the following function defined in $GF(2^8)$.

Let $$J(p) = \begin{cases}
                0 & p = 0\\
                p^{-1} & \text{otherwise}
            \end{cases}
        $$

In [328]:
def J(p):
    if p == 0:
        return p
    else:
        return inv_table[p]

def S(p):
    return T(J(p))

Crucially, $J^2 = I$ and $T^2=I$ (where $I$ is the identity function).  So the inverse of $S$ is $JT$.

In [329]:
def Sinv(q):
    return J(T(q))

In [330]:
S(Sinv(240))

240

AES implements uses S and Sinv as look up tables.

In [331]:
SBox = [S(p) for p in range(0x100)]
SBoxInv = [Sinv(q) for q in range(0x100)]

In [332]:
SBox[SBoxInv[69]]

69

## AES States

AES performs a block cipher that transforms 128 bits into 128 bits.

Internally, the block of 128 bits is organized into a $4\times 4$ array of bytes:

```
s_00   s_01   s_02   s_03
s_10   s_11   s_12   s_13
s_20   s_21   s_22   s_23
s_30   s_31   s_32   2_33
```

All the mathematics we have collected above is used to transform the block in possible ways:

1. Each $s_{ij}$ is substituted using SBox. Inverse uses SBoxInv.
2. Each column, interpreted as an element of $GF(2^8)[x]/x^4+1$, is multiplied by $𝑒𝑥^3+9𝑥^2+𝑑𝑥+𝑏$. The inverse is given by multiplying by $3x^3+x^2+x+2$.
3. Each row is interpreted as an element of $GF(2^8)[x]/x^4+1$. Row $i$ is multiplied by $x^i$.
The effect is to rotate row $i$ by $i$ bytes. The inverse multiplies row i by $x^{4-i}$.
4. Each column $j$, interpreted as an element of $GF(2^8)/x^4+1$, has an associated "key schedule" value $w_i$. This is added to the column. Because $w_i+w_i = 0$ (in this field), this operation is its own inverse.

We take these one at a time.

Operations 2 and 4 will be faster if the state is actually reprented as an array of columns, not as an array of rows. And although 3 appears to be an operation in the ring $GF(2^8)[x]/x^4+1$, it is really just rotating the elements in a row. So 3 will get very little advantage by storing the state as an array of rows. So we choose to think of a state as consisting of columns ```s = [s_0, s_1, s_2, s_3]``` and each entry is an array of four bytes. 


In [333]:
def SubstBytes(s):
    for c in range(4):
        for c in range(4):
            s[c][r] = SBox[s[c][r]]

def SustBytesInv(s):
    for c in range(4):
        for r in range(4):
            s[c][r] = SBoxInv[s[c][r]]

In [334]:
def MixColumns(s):
    for c in range(4):
        s[c] = AES_mult(mixing_p, s[c])
        
def MixColumnsInv(s):
    for c in range(4):
        s[c] = AES_poly_mult(unmixing_p, s[c])

In [368]:
def ShiftRows(s):
    
    for r in range(1,4): # We do nothing to the 0th row, so skip that
        s[0][r],s[1][r],s[2][r],s[3][r] = \
        s[r][r], s[(r+1)&0x3][r], s[(r+2)&0x3][r], s[(r+3)&0x3][r]

In [369]:
def AddRoundKey(s,w):
    for c in range(4):
        for r in range(4):
            s[c][r] ^= w[r]